# <center>Лабораторная работа 3</center>

##### Фирфаров А.С.
##### М8О-408Б

Решить краевую задачу для дифференциального уравнения эллиптического типа. Аппроксимацию уравнения произвести с использованием центрально-разностной схемы. Для решения дискретного аналога применить следующие методы: метод простых итераций (метод Либмана), метод Зейделя, метод простых итераций с верхней релаксацией. Вычислить погрешность численного решения путем сравнения результатов с приведенным в задании аналитическим решением  $U(x, y)$. Исследовать зависимость погрешности от сеточных параметров  $h_x, h_y$.

Общий вид задачи:

$\dfrac{d^2u}{dx^2} + \dfrac{d^2u}{dy^2} = b_x\dfrac{du}{dx} + b_y\dfrac{du}{dy} + cu$ <br>

$\alpha_{1}\dfrac{du}{dx}(0, y) + \beta_{1}u(0, y) = \phi_{1}(y)$ <br>
$\alpha_{2}\dfrac{du}{dx}(l_x, y) + \beta_{1}u(l_x, y) = \phi_{2}(y)$ <br>
$\alpha_{3}\dfrac{du}{dy}(x, 0) + \beta_{3}u(x, 0) = \phi_{3}(x)$ <br>
$\alpha_{4}\dfrac{du}{dy}(x, l_y) + \beta_{3}u(x, l_y) = \phi_{4}(x)$ <br>

Подставим аппроксимации в основное уравнение

$\dfrac{u_{i-1,j} - 2u_{i,j} + u_{i+1,j}}{h^2_x} + \dfrac{u_{i,j-1} - 2u_{i,j} + u_{i,j+1}}{h^2_y} = 
b_x\dfrac{u_{i+1,j} - u_{i-1,j}}{2h_x} + b_y\dfrac{u_{i,j+1} - u_{i,j_1}}{2h_y} + cu_{i,j}$

Домножим на $2h^2_xh^2_y$, сгруппируем коэффициенты при $u_{i,j}$ и получим формулу для вычисления $u^{k+1}_{i,j}$

 $u^{k+1}_{i,j} = \dfrac{b_xh_xh^2_y(u_{i+1,j} - u_{i-1,j}) + b_yh_yh^2_x(u_{i,j+1} - u_{i,j-1}) - 
 2h^2_y(u_{i-1,j} + u_{i+1,j}) - 2h^2_x(u_{i,j-1} + u_{i,j+1})}{-4h^2_y - 4h^2_x - 2h^2_xh^2_yc}$


Для аппроксимации производных в граничных условиях используется двухточечная аппроксимация с 1 порядком точности

$(h_x\beta_1 - \alpha_1)u_{0,j} + \alpha_1u_{1,j} = h_x\phi_1(y)$ <br>
$-\alpha_2u_{N_x-1,j} + (\alpha_2 + h_x\beta_2)u_{N_x,j} = h_x\phi_2(y)$ <br>
$(h_y\beta_3 - \alpha_3)u_{i,0} + \alpha_3u_{i,1} = h_y\phi_3(x)$ <br>
$-\alpha_4u_{i,N_y - 1} + (\alpha_4 + h_y\beta_4)u_{i,N_y} = h_y\phi_4(x)$ <br>

В методе простых итераций для вычисления $u^{k+1}_{i,j}$ используются значения с предыдущей итерации

В методе Зейделя для вычисления $u^{k+1}_{i,j}$ по возможности используются значения, уже вычесленные на $k+1$ итерации

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact, interactive, fixed, interact_manual
import copy
import ipywidgets as widgets

In [ ]:
params = {
    'bx': -2.0,
    'by': -2.0,
    'c': -4.0,
    
    'alpha_x0': 0.0,
    'beta_x0': 1.0,
    'f_phi_x0': lambda y: np.exp(-y) * np.cos(y),
    
    'alpha_xl': 0.0,
    'beta_xl': 1.0,
    'f_phi_xl': lambda y: 0.0,
    
    'alpha_y0': 0.0,
    'beta_y0': 1.0,
    'f_phi_y0': lambda x: np.exp(-x) * np.cos(x),
    
    'alpha_yl': 0.0,
    'beta_yl': 1.0,
    'f_phi_yl': lambda x: 0.0,
    
    'interval_x': (0.0, np.pi/2),
    'interval_y': (0.0, np.pi/2),
    'analytical_solution': lambda x, y: np.exp(-x - y) * np.cos(x) * np.cos(y)
}

In [ ]:
class Task:
    def __init__(self, **params):
        self.bx = params['bx']
        self.by = params['by']
        self.c = params['c']
        
        self.alpha_x0 = params['alpha_x0']
        self.beta_x0 = params['beta_x0']
        self.f_phi_x0 = params['f_phi_x0']
        
        self.alpha_xl = params['alpha_xl']
        self.beta_xl = params['beta_xl']
        self.f_phi_xl = params['f_phi_xl']
        
        self.alpha_y0 = params['alpha_y0']
        self.beta_y0 = params['beta_y0']
        self.f_phi_y0 = params['f_phi_y0']
        
        self.alpha_yl = params['alpha_yl']
        self.beta_yl = params['beta_yl']
        self.f_phi_yl = params['f_phi_yl']

        self.interval_x = params['interval_x']
        self.interval_y = params['interval_y']
        self.analytical_solution = params['analytical_solution']
        
class Solver:
    
    @classmethod
    def solve(cls, task:Task, nx, ny, method, omega=1.0, eps=1e-8, quiet=False):
        hx, hy = cls.__prepare(task.interval_x[0], task.interval_x[1], nx, 
                                 task.interval_y[0], task.interval_y[1], ny)
        if method == 'simple_iterations':
            u = cls.__simple_iterations_method(task, nx, ny, hx, hy, eps, quiet)
        elif method == 'seidel':
            u = cls.__seidel_method(task, nx, ny, hx, hy, 1.0, eps, quiet)
        elif method == 'top_relaxation':
            u = cls.__seidel_method(task, nx, ny, hx, hy, omega, eps, quiet)
        x, y = cls.__get_split_by_space(hx, nx, hy, ny)
        return u, x, y
            
    @classmethod        
    def __simple_iterations_method(cls, task:Task, nx, ny, hx, hy, eps, quiet):
        u_new = np.ones((nx, ny))
        u = np.ones((nx, ny))
        iter = 0
        while True:
            for i in range(1, nx - 1):
                for j in range(1, ny - 1):
                    u_new[i][j] = ((u[i + 1][j] - u[i - 1][j]) * task.bx * hx * hy ** 2 + \
                                  (u[i][j + 1] - u[i][j - 1]) * task.by * hy * hx ** 2 - \
                                   2 * hy ** 2 * (u[i - 1][j] + u[i + 1][j]) - \
                                   2 * hx ** 2 * (u[i][j - 1] + u[i][j + 1])) / \
                                  (-4 * hy ** 2 - 4 * hx ** 2 - 2 * hx ** 2 * hy ** 2 * task.c)
            for j in range(ny):
                u_new[0][j] = (hx * task.f_phi_x0(j * hy) - task.alpha_x0 * u_new[1][j]) / (hx * task.beta_x0 - task.alpha_x0)
                u_new[-1][j] = (hx * task.f_phi_xl(j * hy) + task.alpha_xl * u_new[-2][j]) / (hx * task.beta_xl + task.alpha_xl)
            for i in range(nx):
                u_new[i][0] = (hy * task.f_phi_y0(i * hx) - task.alpha_y0 * u_new[i][1]) / (hy * task.beta_y0 - task.alpha_y0)
                u_new[i][-1] = (hy * task.f_phi_yl(i * hx) + task.alpha_yl * u_new[i][-2]) / (hy * task.beta_yl + task.alpha_yl)
            eps_k = max([max(np.absolute(u_new[i] - u[i])) for i in range(nx)])
            iter += 1
            if eps_k < eps:
                if not quiet:
                    print('Итераций: ', iter)
                return u_new
            u = copy.deepcopy(u_new)
            
    @classmethod        
    def __seidel_method(cls, task:Task, nx, ny, hx, hy, omega, eps, quiet):
        u_prev = np.ones((nx, ny))
        u = np.ones((nx, ny))
        iter = 0
        while True:
            for i in range(1, nx - 1):
                for j in range(1, ny - 1):
                    u[i][j] = ((u[i + 1][j] - u[i - 1][j]) * task.bx * hx * hy ** 2 + \
                              (u[i][j + 1] - u[i][j - 1]) * task.by * hy * hx ** 2 - \
                               2 * hy ** 2 * (u[i - 1][j] + u[i + 1][j]) - \
                               2 * hx ** 2 * (u[i][j - 1] + u[i][j + 1])) / \
                               (-4 * hy ** 2 - 4 * hx ** 2 - 2 * hx ** 2 * hy ** 2 * task.c)
            for j in range(ny):
                u[0][j] = (hx * task.f_phi_x0(j * hy) - task.alpha_x0 * u[1][j]) / (hx * task.beta_x0 - task.alpha_x0)
                u[-1][j] = (hx * task.f_phi_xl(j * hy) + task.alpha_xl * u[-2][j]) / (hx * task.beta_xl + task.alpha_xl)
            for i in range(nx):
                u[i][0] = (hy * task.f_phi_y0(i * hx) - task.alpha_y0 * u[i][1]) / (hy * task.beta_y0 - task.alpha_y0)
                u[i][-1] = (hy * task.f_phi_yl(i * hx) + task.alpha_yl * u[i][-2]) / (hy * task.beta_yl + task.alpha_yl)
            for i in range(nx):
                for j in range(ny):
                    u[i][j] = omega * u[i][j] + (1.0 - omega) * u_prev[i][j]
            eps_k = max([max(np.absolute(u[i] - u_prev[i])) for i in range(nx)])
            iter += 1
            if eps_k < eps:
                if not quiet:
                    print('Итераций: ', iter)
                return u
            u_prev = copy.deepcopy(u)
            
    @classmethod
    def analytical_solution(cls, task:Task, nx, ny):
        hx, hy = cls.__prepare(task.interval_x[0], task.interval_x[1], nx, 
                               task.interval_y[0], task.interval_y[1], ny)
        res = np.zeros((nx, ny))
        for i in range(nx):
            for j in range(ny):
                res[i][j] = task.analytical_solution(hx * i, hy * j)
        return res
    
    @classmethod
    def __get_split_by_space(cls, hx, nx, hy, ny):
        return np.array([hx * i for i in range(nx)]), np.array([hy * j for j in range(ny)])
        
    @classmethod
    def __prepare(cls, begin_x, end_x, nx, begin_y, end_y, ny):
        hx = (end_x - begin_x) / (nx - 1)
        hy = (end_y - begin_y) / (ny - 1)
        return hx, hy

In [ ]:
style = {'description_width': 'initial'}

method_dropdown = widgets.Dropdown(
    options=['simple_iterations', 'seidel', 'top_relaxation'],
    value = 'simple_iterations',
    description='Метод решения',
    style=style
)
nx_ = widgets.BoundedIntText(
    value=30,
    min=5,
    max=500,
    step=1,
    description='nx',
    style=style
)
ny_ = widgets.BoundedIntText(
    value=30,
    min=5,
    max=500,
    step=1,
    description='ny',
    style=style
)
omega_ = widgets.BoundedFloatText(
    value=1.7,
    min=1.0,
    max=2.0,
    step=0.1,
    description='omega',
    style=style
)

display(method_dropdown)
display(nx_)
display(ny_)
display(omega_)

In [ ]:
task = Task(**params)

method = method_dropdown.value
nx = nx_.value
ny = ny_.value
omega = omega_.value

u,x_p,y_p = Solver.solve(task=task, nx=nx, ny=ny, method=method, omega=omega, eps=1e-8)
analytical = Solver.analytical_solution(task=task, nx=nx, ny=ny)

error = max([max(np.absolute(u[i] - analytical[i])) for i in range(len(x_p))])
print('Ошибка = %f' % error)

# Графики решений

In [ ]:
%matplotlib inline

print('method = %s' % method)
print('nx = %s' % nx)
print('ny = %s' % ny)
print('show by x')

def plot_x(x_idx):
    print('x = {0}'.format(x_p[x_idx]))
    plt.figure(figsize=(10, 8))
    plt.plot(y_p, task.analytical_solution(x_p[int(x_idx)], y_p), color='r', label='Точное решение')
    plt.plot(y_p, u[int(x_idx)], color='b', label='Найденное решение')
    plt.xlabel('y')
    plt.ylabel('u(x, y)')
    plt.legend()
    plt.show()

interactive_plot_x = interactive(plot_x, x_idx=(0, len(x_p) - 1, 1))
output_x = interactive_plot_x.children[-1]
output_x.layout.height = '550px'
interactive_plot_x

In [ ]:
%matplotlib inline

print('method = %s' % method)
print('nx = %s' % nx)
print('ny = %s' % ny)
print('show by y')

def plot_y(y_idx):
    u_t = u.transpose()
    print('y = {0}'.format(y_p[y_idx]))
    plt.figure(figsize=(10, 8))
    plt.plot(x_p, task.analytical_solution(x_p, y_p[int(y_idx)]), color='r', label='Точное решение')
    plt.plot(x_p, u_t[int(y_idx)], color='b', label='Найденное решение')
    plt.xlabel('x')
    plt.ylabel('u(x, y)')
    plt.legend()
    plt.show()

interactive_plot_y = interactive(plot_y, y_idx=(0, len(y_p) - 1, 1))
output_y = interactive_plot_y.children[-1]
output_y.layout.height = '550px'
interactive_plot_y

# Графики ошибок

In [ ]:
%matplotlib inline

print('method = %s' % method)
print('nx = %s' % nx)
print('ny = %s' % ny)
print('show by x')

errors_x = np.array([[np.absolute(u[i][j] - analytical[i][j]) for j in range(len(y_p))] for i in range(len(x_p))])
def plot_err_x(x_idx):
    print('x = {0}'.format(x_p[x_idx]))
    plt.figure(figsize=(10, 8))
    plt.plot(y_p, errors_x[int(x_idx)])
    plt.xlabel('y')
    plt.ylabel('u(x, y)')
    plt.show()

interactive_plot_err_x = interactive(plot_err_x, x_idx=(0, len(x_p) - 1, 1))
output_err_x = interactive_plot_err_x.children[-1]
output_err_x.layout.height = '550px'
interactive_plot_err_x

In [ ]:
%matplotlib inline

print('method = %s' % method)
print('nx = %s' % nx)
print('ny = %s' % ny)
print('show by y')

u_tr = u.transpose()
analytical_tr = analytical.transpose()
errors_y = np.array([[np.absolute(u_tr[i][j] - analytical_tr[i][j]) for j in range(len(x_p))] for i in range(len(y_p))])
def plot_err_y(y_idx):
    print('y = {0}'.format(y_p[y_idx]))
    plt.figure(figsize=(10, 8))
    plt.plot(x_p, errors_y[int(y_idx)])
    plt.xlabel('x')
    plt.ylabel('u(x, y)')
    plt.show()

interactive_plot_err_y = interactive(plot_err_y, y_idx=(0, len(y_p) - 1, 1))
output_err_y = interactive_plot_err_y.children[-1]
output_err_y.layout.height = '550px'
interactive_plot_err_y

In [ ]:
n_0 = 10
n_n = 15
steps = np.array([(task.interval_x[1] - task.interval_x[0]) / (n_ - 1) for n_ in range(n_0, n_n)])
m_err = np.zeros(n_n - n_0)
idx = 0

for n_s in range(n_0, n_n):
    u_,x_p_,_ = Solver.solve(task=task, nx=n_s, ny=n_s, method=method, omega=omega, eps=1e-8, quiet=True)
    analytical_ = Solver.analytical_solution(task=task, nx=n_s, ny=n_s)
    max_err = max([max(np.absolute(u_[i] - analytical_[i])) for i in range(len(x_p_))])
    m_err[idx] = max_err
    idx += 1

plt.figure(figsize=(10, 8))
plt.plot(np.log(steps), np.log(m_err))
plt.show()